# List collections

In [24]:
from qdrant_client import QdrantClient

QDRANT_MAIN_URL = "b9d11745-9a5f-46bd-a4f1-50c4f74737ab.us-east-1-0.aws.cloud.qdrant.io:6333"
QDRANT_API_KEY = "xA15yE-Jcv0-Jsmcl_AMzF6UVquNF6D4UL30v3k1GvCipPvEd-2FQQ"

client = QdrantClient(QDRANT_MAIN_URL, api_key=QDRANT_API_KEY)
resp = client.get_collections()
collections = [c.name for c in resp.collections]
collections

['text-embedding-ada-002']

# Create snapshots remotely

In [28]:
QDRANT_NODES = ["node-0-b9d11745-9a5f-46bd-a4f1-50c4f74737ab.us-east-1-0.aws.cloud.qdrant.io:6333"]

snapshot_urls = {}
for collection in collections:
    urls = []
    for node_url in QDRANT_NODES:
        node_client = QdrantClient(node_url, api_key=QDRANT_API_KEY, prefer_grpc=True)
        snapshot_info = node_client.create_snapshot(collection_name=collection)

        snapshot_url = f"{node_url}/collections/{collection}/snapshots/{snapshot_info.name}"
        urls.append(snapshot_url)
        
    snapshot_urls[collection] = urls
    
snapshot_urls

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNIMPLEMENTED
	details = "Received http2 header with status: 404"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Received http2 header with status: 404", grpc_status:12, created_time:"2024-03-31T14:15:07.434651+02:00"}"
>

# Download snapshots locally

In [ ]:
import requests
import os


# Create a directory to store snapshots
os.makedirs("snapshots", exist_ok=True)

local_snapshot_paths = {}
for collection in collections:
    paths = []
    for snapshot_url in snapshot_urls:
        snapshot_name = os.path.basename(snapshot_url)
        local_snapshot_path = os.path.join("snapshots", snapshot_name)

        response = requests.get(snapshot_url, headers={"api-key": QDRANT_API_KEY})
        with open(local_snapshot_path, "wb") as f:
            response.raise_for_status()
            f.write(response.content)

        paths.append(local_snapshot_path)
        
    local_snapshot_paths[collection] = paths

local_snapshot_paths

# Import local snapshots

In [ ]:
NODE_URL = 'https://qdrant.srv.instrukt.ai'

for collection in collections:
    paths = local_snapshot_paths[collection]
    for snapshot_path in paths:
        snapshot_name = os.path.basename(snapshot_path)
        requests.post(
            f"{NODE_URL}/collections/{collection}/snapshots/upload?priority=snapshot",
            headers={
                "api-key": QDRANT_API_KEY,
            },
            files={"snapshot": (snapshot_name, open(snapshot_path, "rb"))},
        )